In [14]:
from quant_utils.db_conn import DB_CONN_JJTG_DATA
from quant_utils.performance import Performance, periods_performance
import pandas as pd
query_sql = """
SELECT
  TradingDay AS TRADE_DT,
  SecuCode AS TICKER_SYMBOL,
  ChangePCT AS RETURN_RATE 
FROM
  jy_indexquote 
WHERE
  1 = 1 
  AND SecuCode IN ( '930950', '930609' ) UNION
SELECT
  END_DATE AS TRADE_DT,
  TICKER_SYMBOL,
  RETURN_RATE 
FROM
  fund_adj_nav 
WHERE
  1 = 1 
  AND TICKER_SYMBOL IN ( "513500" )
"""
df = DB_CONN_JJTG_DATA.exec_query(query_sql)

In [15]:
df = df.pivot_table(index="TRADE_DT", columns="TICKER_SYMBOL", values="RETURN_RATE").dropna()

In [ ]:
df["10%权益"] = df["513500"]*0.05 + 0.05 * df["930950"] + 0.9* df["930609"]
df["20%权益"] = df["513500"]*0.1 + 0.1 * df["930950"] + 0.8* df["930609"]
df["30%权益"] = df["513500"]*0.15 + 0.15 * df["930950"] + 0.8* df["930609"]

df["10%权益_NAV"] = (df["10%权益"]/100 + 1).cumprod()
df["20%权益_NAV"] = (df["20%权益"]/100 + 1).cumprod()
df["30%权益_NAV"] = (df["30%权益"]/100 + 1).cumprod()

In [34]:
col_list = ["10%权益_NAV", "20%权益_NAV", "30%权益_NAV"]
perf_list = []
period_perf_list = []
for col in col_list:
    perf_cls = Performance(df[col])
    perf = perf_cls.stats().T
    perf["组合名称"] = col
    perf_list.append(perf)
    period_perf_temp = perf_cls.rolling_ret_stats()
    period_perf_temp["组合名称"] = col
    period_perf_list.append(period_perf_temp)

perf_df = pd.concat(perf_list)
period_perf_df = pd.concat(period_perf_list)

D:\FundResearch\quant_utils\performance.py:601: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return pd.DataFrame.from_dict(perf_dict).applymap(lambda x: format(x, ".2%"))
D:\FundResearch\quant_utils\performance.py:601: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return pd.DataFrame.from_dict(perf_dict).applymap(lambda x: format(x, ".2%"))
D:\FundResearch\quant_utils\performance.py:601: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return pd.DataFrame.from_dict(perf_dict).applymap(lambda x: format(x, ".2%"))


In [36]:
with pd.ExcelWriter("d:/结果.xlsx") as writer:
    perf_df.to_excel(writer, sheet_name="组合绩效")
    period_perf_df.to_excel(writer, sheet_name="组合周期绩效")
    df[col_list].to_excel(writer, sheet_name="净值")